In [160]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib.patches as mpatches
import seaborn as sb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import seaborn as sns


In [194]:
movielens_dir = "../ml-25m/"
df_newMovies = pd.read_csv(movielens_dir+ "newMovies.csv",index_col="movieId")
df_providers = pd.read_csv(movielens_dir+ "providers.csv")
df_providers.provider_id = df_providers.provider_id.apply(str)
df_providers

,movieId,provider_id,provider_name,type
0,1,3,Google Play Movies,rent
1,1,337,Disney Plus,flatrate
2,1,167,Claro video,flatrate
3,1,3,Google Play Movies,buy
4,2,339,Movistar Play,rent
...,...,...,...,...
34770,208945,2,Apple iTunes,rent
34771,208945,3,Google Play Movies,rent
34772,208945,2,Apple iTunes,buy
34773,208945,3,Google Play Movies,buy


In [195]:
df_model_movies = df_newMovies[ "year"].copy() ## df_newMovies['genres'].str.get_dummies(sep='|')
df_genres = df_newMovies['genres'].str.get_dummies(sep='|')
df_model_movies = pd.merge(left=df_model_movies,right=df_genres, left_on='movieId', right_on='movieId')
df_model_movies

,year,?,Acción,Animación,Aventura,Bélica,Ciencia ficción,Comedia,Crimen,Documental,...,Familia,Fantasía,Historia,Misterio,Música,Película de TV,Romance,Suspense,Terror,Western
movieId,,,,,,,,,,,,,,,,,,,,,
1,1995.0,0,0,1,1,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
2,1995.0,0,0,0,1,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
3,1995.0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
4,1995.0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
5,1995.0,0,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209157,2018.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
209159,2001.0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
209163,2018.0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [196]:
df_model_provider = df_providers.groupby(["movieId"]).agg({'provider_id': "|".join})
df_model_provider = df_model_provider['provider_id'].str.get_dummies(sep='|')
df_model_movies = pd.merge(left=df_model_movies,right=df_model_provider, left_on='movieId', right_on='movieId')
df_model_movies.columns

Index(['year', '?', 'Acción', 'Animación', 'Aventura', 'Bélica',
       'Ciencia ficción', 'Comedia', 'Crimen', 'Documental', 'Drama',
       'Familia', 'Fantasía', 'Historia', 'Misterio', 'Música',
       'Película de TV', 'Romance', 'Suspense', 'Terror', 'Western', '11',
       '119', '167', '190', '2', '3', '31', '315', '337', '339', '350', '384',
       '444', '445', '467', '475', '521', '531', '534', '546', '551', '554',
       '567', '569', '575', '619', '67', '8'],
      dtype='object')

In [215]:
df_model_movies.fillna(0)
X = df_model_movies[[ '?', 'Acción', 'Animación', 'Aventura', 'Bélica',
       'Ciencia ficción', 'Comedia', 'Crimen', 'Documental', 'Drama',
       'Familia', 'Fantasía', 'Historia', 'Misterio', 'Música',
       'Película de TV', 'Romance', 'Suspense', 'Terror', 'Western', '11',
       '119', '167', '190', '2', '3', '31', '315', '337', '339', '350', '384',
       '444', '445', '467', '475', '521', '531', '534', '546', '551', '554',
       '567', '569', '575', '619', '67', '8']].values.astype(np.float)
X_train, X_test = train_test_split(X)
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
##X_test = scaler.transform(X_test)
##knn = NearestNeighbors(metric='cosine', algorithm='brute')
##knn.fit(X_train)


In [216]:
X_train

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [220]:
knn = NearestNeighbors(metric='cosine', algorithm='brute')
knn.fit(X_train)
distances, indices = knn.kneighbors(X_train, n_neighbors=10)


In [221]:
indices

array([[3928,    0,  537, ..., 5383,  350, 2806],
       [   1, 3523, 2426, ..., 8301, 5463, 3883],
       [   2, 4515,  487, ..., 1836, 5775, 7887],
       ...,
       [3952, 9085, 1319, ..., 7164, 8334, 2543],
       [5044, 5974, 1640, ..., 1981,  344, 1927],
       [9087, 6944, 3674, ..., 3253, 4868, 2843]], dtype=int64)